In [46]:
import openfe
from openfe_skunkworks.protocols import openmm_afe

In [47]:
# first load chemistry components
solvent = openfe.SolventComponent(smiles='O')
benzene = openfe.SmallMoleculeComponent.from_sdf_file('./benzene.sdf')

# arrange into ChemicalSystems that represent the free energy difference
cs_solvent = openfe.ChemicalSystem({'solvent': solvent}, name='solvent system')
cs_benzene = openfe.ChemicalSystem({'solvent': solvent, 'ligand': benzene}, name='solvated benzene')

In [48]:
# grab default Settings for the AFE solvation protocol
settings = openmm_afe.AbsoluteSolvationProtocol.default_settings()

In [49]:
# in gromacs currency, this is a little like the .mdp file
settings

{'alchemical_settings': {},
 'integrator_settings': {'barostat_frequency': <Quantity(25, 'timestep')>,
                         'constraint_tolerance': 1e-06,
                         'langevin_collision_rate': <Quantity(1.0, '1 / picosecond')>,
                         'n_restart_attempts': 20,
                         'reassign_velocities': False,
                         'remove_com': False,
                         'timestep': <Quantity(4, 'femtosecond')>},
 'lambda_settings': {'lambda_elec': [0.0,
                                     0.25,
                                     0.5,
                                     0.75,
                                     1.0,
                                     1.0,
                                     1.0,
                                     1.0,
                                     1.0,
                                     1.0,
                                     1.0,
                                     1.0,
                            

In [50]:
# notably, this settings section is an OpenMMSolvationSettings
print(type(settings.solvation_settings))

<class 'openfe.protocols.openmm_utils.omm_settings.OpenMMSolvationSettings'>


In [51]:
# I'm imagining that we'll have something like this...
from openff.models.models import BaseModel

class PackmolSolvationSettings(BaseModel):
    tolerance: float=2.0

pss = PackmolSolvationSettings()

In [52]:
# this currently fails, we'll either have to:
# put a solvation_settings: Union[OpenMMSolvationSettings, OpenMMSolvationSettings]
# OR create an empty base class that both the above classes inherit from, not sure which is cleaner w/ pydantic
settings.solvation_settings = pss

# we can then replace the "if interchange:" line with "if isinstance(solvation_settings, PackmolSolvationSettings):"

ValidationError: 1 validation error for AbsoluteSolvationSettings
solvation_settings -> tolerance
  extra fields not permitted (type=value_error.extra)

In [53]:
# crystallise the settings into a Protocol, a chemistry-less definition of a algorithm
protocol = openmm_afe.AbsoluteSolvationProtocol(settings)

In [54]:
# attach the chemistry by bundling the two together into a Transformation
transformation = openfe.Transformation(
    stateA=cs_benzene,
    stateB=cs_solvent,
    mapping=None,
    protocol=protocol
)

In [55]:
transformation

Transformation(stateA=ChemicalSystem(name=solvated benzene, components={'solvent': SolventComponent(name=O, Na+, Cl-), 'ligand': SmallMoleculeComponent(name=benzene)}), stateB=ChemicalSystem(name=solvent system, components={'solvent': SolventComponent(name=O, Na+, Cl-)}), protocol=<AbsoluteSolvationProtocol-08b4951f9786bdc77ff4c1d9e9f63c90>)

In [56]:
# this is a bit like a gromacs .tpr, it's a self contained simulation definition ready to go
transformation.dump('benzene.json')

In [57]:
# running "openfe quickrun benzene.json" should start the simulation